## Case Study - Freddie Mac Bonds Alternate File

In [1]:
import warnings

import numpy as np
import pandas as pd
from scipy.stats import norm
from scipy.optimize import fsolve

warnings.filterwarnings("ignore")

In [2]:
# Path to Excel file containing required callable bond and discount curve data
alternate_bond_file = '../data/callable_bonds_2025-02-18.xlsx'
discount_file = '../data/discount_curve_2025-02-13.xlsx'

# Read the Excel file for info and quotes of callable bonds into a pandas DataFrame
callable_bonds_info = pd.read_excel(alternate_bond_file, sheet_name=0)
callable_bonds_quotes = pd.read_excel(alternate_bond_file, sheet_name=1)

# Read the Excel file for discount curve into a pandas DataFrame
discount_curve = pd.read_excel(discount_file, sheet_name=0)

# Display the DataFrame
display(callable_bonds_info.style.set_caption("Callable Bonds Info"))
display(callable_bonds_quotes.style.set_caption("Callable Bonds Quotes"))
display(discount_curve.head().style.set_caption("Discount Curve"))

,info,FHLMC 4.55 02/11/28
0,CUSIP,3134HA6A6
1,Issuer,FREDDIE MAC
2,Maturity Type,CALLABLE
3,Issuer Industry,GOVT AGENCY
4,Amount Issued,1000000000
5,Cpn Rate,0.045500
6,Cpn Freq,2
7,Date Quoted,2025-02-18 00:00:00
8,Date Issued,2025-02-11 00:00:00
9,Date Matures,2028-02-11 00:00:00


,quotes,FHLMC 4.55 02/11/28
0,Date Quoted,2025-02-18 00:00:00
1,TTM,2.976044
2,Clean Price,99.720000
3,Dirty Price,99.833750
4,Accrued Interest,0.113750
5,YTM Call,4.846113
6,YTM Maturity,4.651418
7,Duration,2.812765
8,Modified Duration,2.748835
9,Convexity,0.091609


,ttm,maturity date,spot rate,discount
0,0.500000,2025-08-13 00:00:00,0.043743,0.978597
1,1.000000,2026-02-13 00:00:00,0.042890,0.958451
2,1.500000,2026-08-13 00:00:00,0.042238,0.939228
3,2.000000,2027-02-13 00:00:00,0.041843,0.920515
4,2.500000,2027-08-13 00:00:00,0.041632,0.902117


## 1. Pricing the Callable Bond

### 1.1

In [3]:
# Function to interpolate discount factors
def interpolate_discount_factor(ttm_target, discount_curve):
    """
    Interpolate discount factor for a given time to maturity.
    Uses linear interpolation on log of discount factors.
    """
    ttm_curve = discount_curve['ttm'].values
    discount_factors = discount_curve['discount'].values
    
    if ttm_target <= ttm_curve[0]:
        return np.interp(ttm_target, ttm_curve, discount_factors)
    elif ttm_target >= ttm_curve[-1]:
        return np.interp(ttm_target, ttm_curve, discount_factors)
    else:
        # Linear interpolation on log(discount)
        log_discounts = np.log(discount_factors)
        log_df_interpolated = np.interp(ttm_target, ttm_curve, log_discounts)
        return np.exp(log_df_interpolated)


# Generate cash flow schedule for a bond
def generate_cash_flows(date_start, date_end, coupon_rate, coupon_freq, principal=100):
    """
    Generate cash flow dates and amounts for a bond.
    
    Parameters:
    - date_start: valuation date
    - date_end: maturity date
    - coupon_rate: annual coupon rate
    - coupon_freq: coupons per year
    - principal: face value (default 100)
    
    Returns:
    - DataFrame with columns: date, time_to_cf, coupon_payment, principal_payment, total_cf
    """
    # Create coupon payment dates
    cf_dates = []
    current_date = date_end
    
    while current_date > date_start:
        cf_dates.append(current_date)
        current_date = current_date - pd.DateOffset(months=int(12/coupon_freq))
    
    cf_dates.reverse()
    
    # Calculate time to each cash flow (in years)
    times_to_cf = [(d - date_start).days / 365.25 for d in cf_dates]
    
    # Calculate coupon payments
    coupon_payment = principal * coupon_rate / coupon_freq
    coupon_payments = [coupon_payment] * len(cf_dates)
    
    # Principal payment at maturity
    principal_payments = [0] * (len(cf_dates) - 1) + [principal]
    
    # Total cash flows
    total_cfs = [c + p for c, p in zip(coupon_payments, principal_payments)]
    
    df = pd.DataFrame({
        'date': cf_dates,
        'time_to_cf': times_to_cf,
        'coupon_payment': coupon_payments,
        'principal_payment': principal_payments,
        'total_cf': total_cfs
    })
    
    return df


# Price a bond using discount curve
def price_bond(cash_flows_df, discount_curve):
    """
    Price a bond using cash flows and discount curve.
    
    Parameters:
    - cash_flows_df: DataFrame with 'time_to_cf' and 'total_cf' columns
    - discount_curve: DataFrame with 'ttm' and 'discount' columns
    
    Returns:
    - bond_price: present value of all cash flows
    """
    pv = 0
    for _, row in cash_flows_df.iterrows():
        ttm = row['time_to_cf']
        cf = row['total_cf']
        df = interpolate_discount_factor(ttm, discount_curve)
        pv += cf * df
    
    return pv    


# Extract bond information for FHLMC 4.55 02/11/28
bond_idx = 1
bond_name = 'FHLMC 4.55 02/11/28'

# Bond characteristics
coupon_rate = callable_bonds_info.iloc[5, bond_idx]
coupon_freq = callable_bonds_info.iloc[6, bond_idx]
date_quoted = pd.to_datetime(callable_bonds_info.iloc[7, bond_idx])
date_issued = pd.to_datetime(callable_bonds_info.iloc[8, bond_idx])
date_matures = pd.to_datetime(callable_bonds_info.iloc[9, bond_idx])
date_first_call = pd.to_datetime(callable_bonds_info.iloc[10, bond_idx])
date_next_call = pd.to_datetime(callable_bonds_info.iloc[11, bond_idx])
strike = callable_bonds_info.iloc[12, bond_idx]

# Market quotes
ttm = callable_bonds_quotes.iloc[1, bond_idx]
clean_price = callable_bonds_quotes.iloc[2, bond_idx]
dirty_price = callable_bonds_quotes.iloc[3, bond_idx]
accrued_interest = callable_bonds_quotes.iloc[4, bond_idx]
quoted_duration = callable_bonds_quotes.iloc[7, bond_idx]
implied_vol = callable_bonds_quotes.iloc[12, bond_idx]


# Display bond characteristics
bond_info_dict = {
    "Bond": [bond_name],
    "Coupon Rate": [f"{coupon_rate:.4f} ({coupon_rate*100:.2f}%)"],
    "Coupon Frequency": [f"{coupon_freq} (semi-annual)"],
    "Date Quoted": [date_quoted.date()],
    "Date Matures": [date_matures.date()],
    "Date Next Call": [date_next_call.date()],
    "Strike": [strike],
    "TTM (years)": [f"{ttm:.4f}"],
    "Market Dirty Price": [f"{dirty_price:.6f}"]
}

bond_info_df = pd.DataFrame(bond_info_dict)
display(bond_info_df.style.set_caption("FHLMC 4.55 02/11/28 Bond Characteristics"))

,Bond,Coupon Rate,Coupon Frequency,Date Quoted,Date Matures,Date Next Call,Strike,TTM (years),Market Dirty Price
0,FHLMC 4.55 02/11/28,0.0455 (4.55%),2 (semi-annual),2025-02-18,2028-02-11,2026-02-11,100,2.9760,99.833750


In [4]:
# 1. Market price (callable bond)
callable_bond_price = dirty_price

# 2. Hypothetical non-callable bond (full maturity)
cf_hypothetical_full = generate_cash_flows(
    date_quoted, date_matures, coupon_rate, coupon_freq,
)
price_hypothetical_full = price_bond(cf_hypothetical_full, discount_curve)

# 3. Hypothetical non-callable bond (call date maturity)
cf_hypothetical_call = generate_cash_flows(
    date_quoted, date_next_call, coupon_rate, coupon_freq,
)
price_hypothetical_call = price_bond(cf_hypothetical_call, discount_curve)

# Organize results in a DataFrame for cleaner display
pricing_summary = pd.DataFrame(
    {
        "Scenario": [
            "Market Dirty Price (Callable Bond)",
            f"Hypothetical Non-Callable (to Maturity {date_matures.date()})",
            f"Hypothetical Non-Callable (to Call Date {date_next_call.date()})",
        ],
        "Model Price": [
            f"{callable_bond_price:.6f}",
            f"{price_hypothetical_full:.6f}",
            f"{price_hypothetical_call:.6f}",
        ],
    }
)

display(pricing_summary.style.set_caption(f"Bond pricing summary - {bond_name}"))


# Market Implied Call Value = Hypothetical Non-Callable (to Maturity) - Callable Bond Price
market_implied_call_value = price_hypothetical_full - callable_bond_price

call_value_df = pd.DataFrame(
    {
        "": ["Market Implied Call Value"],
        "Value": [f"{market_implied_call_value:.6f}"],
    }
)

display(call_value_df.style.set_caption(f"Market Implied Call Value - {bond_name}"))

,Scenario,Model Price
0,Market Dirty Price (Callable Bond),99.833750
1,Hypothetical Non-Callable (to Maturity 2028-02-11),101.194935
2,Hypothetical Non-Callable (to Call Date 2026-02-11),100.332854


,,Value
0,Market Implied Call Value,1.361185


### 1.2

In [5]:
# Calculate time to call date in years
time_to_call = (date_next_call - date_quoted).days / 365

# Get the discount factor to the call date (from today)
discount_to_call = interpolate_discount_factor(time_to_call, discount_curve)

# Generate cash flows occurring from call date to final maturity
cf_from_call_to_maturity = generate_cash_flows(
    date_next_call, date_matures, coupon_rate, coupon_freq
)

# Calculate time (in years) from call date for each cash flow
cf_from_call_to_maturity["time_from_call"] = [
    (d - date_next_call).days / 365 for d in cf_from_call_to_maturity["date"]
]

# Initialize present value at call date
price_at_call_date = 0.0

# Loop through each projected cash flow and discount back to the call date
for _, row in cf_from_call_to_maturity.iterrows():
    cf = row["total_cf"]
    time_from_call = row["time_from_call"]
    time_from_quote = time_to_call + time_from_call  # years from today to cash flow

    # Discount factor from today to cash flow
    df_from_quote = interpolate_discount_factor(time_from_quote, discount_curve)
    # Discount factor from call date to cash flow (forward discounting)
    df_from_call = df_from_quote / discount_to_call

    price_at_call_date += cf * df_from_call

# Forward price equals projected price at call date (already discounted appropriately)
forward_price = price_at_call_date

# Organize results in a DataFrame for display
forward_summary_df = pd.DataFrame(
    {
        "Parameter": [
            "Time to Call Date (years)",
            "Discount Factor to Call Date",
            "Forward Price F(0, T1)",
        ],
        "Value": [
            f"{time_to_call:.6f}",
            f"{discount_to_call:.6f}",
            f"{forward_price:.6f}",
        ],
    }
)

display(
    forward_summary_df.style.set_caption(
        f"Forward Price Calculation to Call Date - {bond_name}"
    )
)

,Parameter,Value
0,Time to Call Date (years),0.980822
1,Discount Factor to Call Date,0.959216
2,"Forward Price F(0, T1)",100.893398


### 1.3

In [6]:
# Extract the spot rate at time T1 (call date)
spot_rate_at_call = interpolate_discount_factor(time_to_call, discount_curve)
# Convert to rate: r = -ln(Z(T))/T
forward_rate_approx = -np.log(spot_rate_at_call) / time_to_call

# Convert implied vol from rate to price
sigma_rate = implied_vol / 100  # Convert from percentage
duration = quoted_duration  # Use quoted duration

sigma_price = duration * sigma_rate * forward_rate_approx

# Organize results in a DataFrame for display
implied_vol_conversion_df = pd.DataFrame(
    {
        "Parameter": [
            "Time to Call Date (years)",
            "Discount Factor at Call",
            "Approximate Forward Rate f(T1)",
            "Implied Vol (rate)",
            "Quoted Duration",
            "Implied Vol (price)",
        ],
        "Value": [
            f"{time_to_call:.6f}",
            f"{spot_rate_at_call:.6f}",
            f"{forward_rate_approx:.6f} ({forward_rate_approx*100:.4f}%)",
            f"{implied_vol:.4f}% = {sigma_rate:.6f}",
            f"{duration:.6f}",
            f"{sigma_price:.6f} = {sigma_price*100:.4f}%",
        ],
    }
)

display(implied_vol_conversion_df.style.set_caption("Implied Volatility Conversion"))

,Parameter,Value
0,Time to Call Date (years),0.980822
1,Discount Factor at Call,0.959216
2,Approximate Forward Rate f(T1),0.042453 (4.2453%)
3,Implied Vol (rate),23.5455% = 0.235455
4,Quoted Duration,2.812765
5,Implied Vol (price),0.028116 = 2.8116%


### 1.4

In [7]:
# Black's formula for call option
def blacks_call_option(F, K, sigma, T, Z):
    """
    Calculate call option value using Black's formula.

    Parameters:
    - F: forward price
    - K: strike price
    - sigma: volatility of forward price
    - T: time to expiration
    - Z: discount factor to expiration

    Returns:
    - call_value: value of call option
    - d1: Black-Scholes d1 parameter
    - d2: Black-Scholes d2 parameter
    """
    if T <= 0 or sigma <= 0:
        call_val = max(F - K, 0) * Z
        d1 = float('nan')
        d2 = float('nan')
        return call_val, d1, d2

    d1 = (np.log(F / K) + 0.5 * sigma**2 * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    call_value = Z * (F * norm.cdf(d1) - K * norm.cdf(d2))
    return call_value, d1, d2


# Calculate call option value
F = forward_price
K = strike
sigma = sigma_price
T = time_to_call
Z = discount_to_call

call_value, d1, d2 = blacks_call_option(F, K, sigma, T, Z)

# Option analytics dataframe
black_formula_df = pd.DataFrame(
    {
        "Parameter": [
            "Forward Price (F)",
            "Strike Price (K)",
            "Volatility (σ)",
            "Volatility (%)",
            "Time to Call (T, years)",
            "Discount Factor Z(0,T)",
            "d1",
            "d2",
            "N(d1)",
            "N(d2)",
            "Call Option Value",
        ],
        "Value": [
            f"{F:.6f}",
            f"{K:.6f}",
            f"{sigma:.6f}",
            f"{sigma*100:.4f}%",
            f"{T:.6f}",
            f"{Z:.6f}",
            f"{d1:.6f}",
            f"{d2:.6f}",
            f"{norm.cdf(d1):.6f}",
            f"{norm.cdf(d2):.6f}",
            f"{call_value:.6f}"
        ],
    }
)
display(black_formula_df.style.set_caption("Black's Formula - Embedded Call Option"))

# Calculate callable bond value and comparison
price_callable_model = price_hypothetical_full - call_value

callable_bond_valuation_df = pd.DataFrame(
    {
        "Parameter": [
            "Non-Callable Bond Price",
            "Embedded Call Value",
            "Callable Bond Price (Model)",
            "Market Price",
            "Model - Market",
            "Percentage Difference",
        ],
        "Value": [
            f"{price_hypothetical_full:.6f}",
            f"{call_value:.6f}",
            f"{price_callable_model:.6f}",
            f"{dirty_price:.6f}",
            f"{price_callable_model - dirty_price:.6f}",
            f"{(price_callable_model - dirty_price)/dirty_price * 100:.4f}%"
        ],
    }
)
display(callable_bond_valuation_df.style.set_caption("Callable Bond Valuation (Model vs Market)"))

,Parameter,Value
0,Forward Price (F),100.893398
1,Strike Price (K),100.000000
2,Volatility (σ),0.028116
3,Volatility (%),2.8116%
4,"Time to Call (T, years)",0.980822
5,"Discount Factor Z(0,T)",0.959216
6,d1,0.333344
7,d2,0.305499
8,N(d1),0.630563
9,N(d2),0.620007


,Parameter,Value
0,Non-Callable Bond Price,101.194935
1,Embedded Call Value,1.552894
2,Callable Bond Price (Model),99.642042
3,Market Price,99.833750
4,Model - Market,-0.191708
5,Percentage Difference,-0.1920%


### 1.5

In [8]:
# Function to calculate YTM
def calculate_ytm(price, cash_flows_df, coupon_freq, initial_guess=0.04):
    """
    Calculate yield to maturity given price and cash flows.

    Parameters:
    - price: dirty price of the bond
    - cash_flows_df: DataFrame with 'time_to_cf' and 'total_cf'
    - coupon_freq: number of coupons per year
    - initial_guess: initial guess for YTM

    Returns:
    - ytm: yield to maturity (annual)
    """
    from scipy.optimize import fsolve

    def price_difference(ytm):
        pv = 0.0
        for _, row in cash_flows_df.iterrows():
            t = row["time_to_cf"]
            cf = row["total_cf"]
            pv += cf / (1 + ytm / coupon_freq) ** (t * coupon_freq)
        return pv - price

    ytm = fsolve(price_difference, initial_guess)[0]
    return ytm

# Prepare values as decimals
quoted_ytm_maturity = callable_bonds_quotes.iloc[6, bond_idx] / 100
quoted_ytm_call = callable_bonds_quotes.iloc[5, bond_idx] / 100

# YTM - Never Called (full maturity)
ytm_never_called = calculate_ytm(price_hypothetical_full, cf_hypothetical_full, coupon_freq)

# YTM - Certainly Called (call at the earliest possible date)
cf_certainly_called = cf_hypothetical_call.copy()
# Replace the last principal payment with the strike price at the call date
cf_certainly_called.iloc[-1, cf_certainly_called.columns.get_loc("principal_payment")] = strike
cf_certainly_called.iloc[-1, cf_certainly_called.columns.get_loc("total_cf")] = (
    cf_certainly_called.iloc[-1]["coupon_payment"] + strike
)
ytm_certainly_called = calculate_ytm(dirty_price, cf_certainly_called, coupon_freq, initial_guess=0.05)

# Compose summary DataFrame in a clean format
ytm_summary_df = pd.DataFrame(
    {
        "Scenario": ["Never Called (Maturity)", "Certainly Called"],
        "Model YTM (%)": [f"{ytm_never_called*100:.4f}", f"{ytm_certainly_called*100:.4f}"],
        "Quoted YTM (%)": [f"{quoted_ytm_maturity*100:.4f}", f"{quoted_ytm_call*100:.4f}"],
        "Difference (bp)": [
            f"{(ytm_never_called - quoted_ytm_maturity)*10000:.2f}",
            f"{(ytm_certainly_called - quoted_ytm_call)*10000:.2f}",
        ],
    }
)
display(ytm_summary_df.style.set_caption("Yield to Maturity (YTM) Summary: Model vs Market"))


,Scenario,Model YTM (%),Quoted YTM (%),Difference (bp)
0,Never Called (Maturity),4.1541,4.6514,-49.74
1,Certainly Called,4.8205,4.8461,-2.56


### 1.6

In [9]:
# Duration of hypothetical non-callable bond (analytical)
def calculate_duration_analytical(price, cash_flows_df, coupon_freq):
    """
    Calculate duration analytically from cash flows.
    Uses the YTM-based duration formula.
    """
    ytm = calculate_ytm(price, cash_flows_df, coupon_freq)

    # Calculate Macaulay duration
    weighted_time = 0
    pv_total = 0

    for _, row in cash_flows_df.iterrows():
        t = row['time_to_cf']
        cf = row['total_cf']
        pv = cf / (1 + ytm / coupon_freq) ** (t * coupon_freq)
        weighted_time += t * pv
        pv_total += pv

    macaulay_duration = weighted_time / pv_total

    # Modified duration
    modified_duration = macaulay_duration / (1 + ytm / coupon_freq)

    return macaulay_duration, modified_duration

mac_dur_hypo, mod_dur_hypo = calculate_duration_analytical(
    price_hypothetical_full, cf_hypothetical_full, coupon_freq
)

duration_summary_df = pd.DataFrame({
    "Metric": [
        "Macaulay Duration",
        "Modified Duration",
        "Quoted Duration",
        "Difference (Model - Quoted)"
    ],
    "Value": [
        f"{mac_dur_hypo:.6f}",
        f"{mod_dur_hypo:.6f}",
        f"{quoted_duration:.6f}",
        f"{mod_dur_hypo - quoted_duration:.6f}"
    ]
})

display(duration_summary_df.style.set_caption("Hypothetical Non-Callable Bond Duration Summary"))


# Duration of callable bond (numerical)
def calculate_duration_numerical_callable(discount_curve, callable_params, bp_shift=0.0001):
    """
    Calculate duration of callable bond using numerical differentiation.
    Shift spot curve by ±1bp and recalculate bond value.
    
    Returns: modified duration
    """
    # Create shifted curves
    dc_up = discount_curve.copy()
    dc_down = discount_curve.copy()
    
    dc_up['spot rate'] = dc_up['spot rate'] + bp_shift
    dc_down['spot rate'] = dc_down['spot rate'] - bp_shift
    
    # Recalculate discount factors
    dc_up['discount'] = np.exp(-dc_up['spot rate'] * dc_up['ttm'])
    dc_down['discount'] = np.exp(-dc_down['spot rate'] * dc_down['ttm'])
    
    # Price hypothetical bond with shifted curves
    price_hypo_up = price_bond(cf_hypothetical_full, dc_up)
    price_hypo_down = price_bond(cf_hypothetical_full, dc_down)
    
    # Recalculate forward prices and call values
    # For up shift
    discount_to_call_up = interpolate_discount_factor(time_to_call, dc_up)
    price_at_call_up = 0
    for _, row in cf_from_call_to_maturity.iterrows():
        time_from_call = row['time_from_call']
        time_from_quote = time_to_call + time_from_call
        cf = row['total_cf']
        df_from_quote = interpolate_discount_factor(time_from_quote, dc_up)
        df_from_call = df_from_quote / discount_to_call_up
        price_at_call_up += cf * df_from_call
    
    forward_price_up = price_at_call_up
    call_value_up, _, _ = blacks_call_option(forward_price_up, K, sigma, T, discount_to_call_up)
    price_callable_up = price_hypo_up - call_value_up
    
    # For down shift
    discount_to_call_down = interpolate_discount_factor(time_to_call, dc_down)
    price_at_call_down = 0
    for _, row in cf_from_call_to_maturity.iterrows():
        time_from_call = row['time_from_call']
        time_from_quote = time_to_call + time_from_call
        cf = row['total_cf']
        df_from_quote = interpolate_discount_factor(time_from_quote, dc_down)
        df_from_call = df_from_quote / discount_to_call_down
        price_at_call_down += cf * df_from_call
    
    forward_price_down = price_at_call_down
    call_value_down, _, _ = blacks_call_option(forward_price_down, K, sigma, T, discount_to_call_down)
    price_callable_down = price_hypo_down - call_value_down
    
    # Calculate duration
    duration_numerical = -(price_callable_up - price_callable_down) / (2 * bp_shift) / price_callable_model
    
    return duration_numerical, price_callable_up, price_callable_down

duration_callable_numerical, price_up, price_down = calculate_duration_numerical_callable(
    discount_curve, None
)

# Duration DataFrame
duration_df = pd.DataFrame({
    "Value": [
        price_callable_model,
        price_up,
        price_down,
        duration_callable_numerical,
        quoted_duration,
        duration_callable_numerical - quoted_duration
    ]
}, index=[
    "Price (base)",
    "Price (+1bp)",
    "Price (-1bp)",
    "Modified Duration",
    "Quoted Duration",
    "Difference"
])

display(duration_df.style.set_caption("Callable Bond Duration (Numerical)"))


# Summary as DataFrame
duration_summary_df = pd.DataFrame({
    "Model Duration": [mod_dur_hypo, duration_callable_numerical],
    "Quoted": [quoted_duration, quoted_duration],
    "Difference": [mod_dur_hypo - quoted_duration, duration_callable_numerical - quoted_duration]
}, index=["Hypothetical (Non-Call)", "Callable (Numerical)"])

display(
    duration_summary_df.style
    .set_caption("Summary")
    .format("{:6.4f}")
)

,Metric,Value
0,Macaulay Duration,2.817717
1,Modified Duration,2.760383
2,Quoted Duration,2.812765
3,Difference (Model - Quoted),-0.052382


,Value
Price (base),99.642042
Price (+1bp),99.552698
Price (-1bp),99.586199
Modified Duration,1.681087
Quoted Duration,2.812765
Difference,-1.131678


,Model Duration,Quoted,Difference
Hypothetical (Non-Call),2.7604,2.8128,-0.0524
Callable (Numerical),1.6811,2.8128,-1.1317


### 1.7

In [10]:
# Function to calculate callable bond price with OAS
def price_callable_with_oas(oas, discount_curve, market_price):
    """
    Calculate callable bond price with an OAS spread added to the curve.
    Returns the difference from market price for optimization.
    
    Parameters:
    - oas: option-adjusted spread (as decimal, e.g., 0.001 for 10bp)
    - discount_curve: base discount curve
    - market_price: target market price
    
    Returns:
    - price difference (model - market)
    """
    # Create shifted curve
    dc_oas = discount_curve.copy()
    dc_oas['spot rate'] = dc_oas['spot rate'] + oas
    dc_oas['discount'] = np.exp(-dc_oas['spot rate'] * dc_oas['ttm'])
    
    # Price hypothetical bond
    price_hypo_oas = price_bond(cf_hypothetical_full, dc_oas)
    
    # Calculate forward price
    discount_to_call_oas = interpolate_discount_factor(time_to_call, dc_oas)
    price_at_call_oas = 0
    for _, row in cf_from_call_to_maturity.iterrows():
        time_from_call = row['time_from_call']
        time_from_quote = time_to_call + time_from_call
        cf = row['total_cf']
        df_from_quote = interpolate_discount_factor(time_from_quote, dc_oas)
        df_from_call = df_from_quote / discount_to_call_oas
        price_at_call_oas += cf * df_from_call
    
    forward_price_oas = price_at_call_oas
    
    # Calculate call option value
    call_value_oas, _, _ = blacks_call_option(forward_price_oas, K, sigma, T, discount_to_call_oas)
    
    # Calculate callable bond price
    price_callable_oas = price_hypo_oas - call_value_oas
    
    return price_callable_oas - market_price

# Solve for OAS
oas_model = fsolve(price_callable_with_oas, x0=0.0, args=(discount_curve, dirty_price))[0]

# Verify
price_with_oas = price_callable_with_oas(oas_model, discount_curve, dirty_price) + dirty_price

quoted_oas_decimal = callable_bonds_quotes.iloc[11, bond_idx] / 10000
difference_bp = (oas_model - quoted_oas_decimal) * 10000

oas_results_df = pd.DataFrame({
    "Metric": [
        "Model OAS",
        "Quoted OAS",
        "Difference (bp)",
        "Market Price",
        "Model Price (OAS)",
        "Model - Market"
    ],
    "Value": [
        f"{oas_model:.8f} ({oas_model * 10000:.4f} bp)",
        f"{quoted_oas_decimal:.8f} ({callable_bonds_quotes.iloc[11, bond_idx]:.4f} bp)",
        f"{difference_bp:.4f}",
        f"{dirty_price:.6f}",
        f"{price_with_oas:.6f}",
        f"{price_with_oas - dirty_price:.8f}"
    ]
})

display(oas_results_df.set_index("Metric").style.set_caption("Option-Adjusted Spread (OAS)"))

,Value
Metric,
Model OAS,-0.00161290 (-16.1290 bp)
Quoted OAS,-0.00089427 (-8.9427 bp)
Difference (bp),-7.1863
Market Price,99.833750
Model Price (OAS),99.833750
Model - Market,0.00000000
